In [ ]:
import pandas as pd
import numpy as np  
from sklearn.linear_model import LinearRegression
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns  
import plotly.graph_objects as go

# Functions  

def generate_sample_data(num_buildings=3):
    days = pd.date_range('1/1/2023', periods=365)
    data = {}
    
    for n in range(1, num_buildings+1):              
        series = pd.Series(np.random.randint(50, 100, 365), index=days)
        data[f'Building {n}'] = series
        
    return data

def create_models(data):
    models = {}
    
    for name, series in data.items():
        X = np.arange(len(series)).reshape(-1, 1)  
        y = series.values    
        model = LinearRegression().fit(X, y)
        models[name] = model
        
    return models
   
def make_predictions(models, data):
    
    pred_data = {}
    
    for name, model in models.items():
        X_pred = np.arange(len(data[name])).reshape(-1, 1)  
        preds = model.predict(X_pred)
        
        df = pd.DataFrame({
            'Actual': data[name],                          
            'Predicted': preds,
            'Projected': preds  
        })
        
        pred_data[name] = df
        
    return pred_data

suggestions = {
   'Building 1': [
      'Reduce AC use',
      'Upgrade air filters'
   ],
   'Building 2': [
      'Install LED lighting',
      'Add occupancy sensors'  
   ]
}
    
def adjust_predictions(building, weight, pred_data):
    pred_data[building]['Predicted'] = pred_data[building]['Predicted'] * weight 
    pred_data[building]['Projected'] = pred_data[building]['Predicted'] * weight
    
def implement_changes(building, percent, pred_data):
    pred_data[building]['Projected'] = pred_data[building]['Predicted'] * (1 - percent/100)
    
# Streamlit app

st.title('Energy Dashboard')
 
data = generate_sample_data(2)
models = create_models(data)
pred_data = make_predictions(models, data)  

building = st.selectbox('Select building:', list(data.keys()))  

cols = ['Actual', 'Predicted', 'Projected']

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))

# Plot charts
pred_data[building][cols].plot(ax=axes[0,0], title='Actual vs Predicted')  
st.pyplot(fig)

weight = st.slider('Efficiency change: ', 0.5, 1.5, 1.0, 0.1)
percent = st.slider('Projected savings (%): ', 0, 30, 10, 5)  

adjust_predictions(building, weight, pred_data)  
implement_changes(building, percent, pred_data)

radar = go.Figure(data=[go.Scatterpolar(
      r = pred_data[building][cols].mean().values,
      theta = cols)])

st.plotly_chart(radar)

savings = pred_data[building]['Actual'].sum() 
st.write(f'Predicted savings: {savings:.2f} kWh')

if building in suggestions:
  st.header('Suggestions')
  for s in suggestions[building]:
    st.write('- ' + s)